## Read STATA Data

In [635]:
import pandas

data = pandas.read_stata("CA.collapsed.dta")
#print data.describe()
print data.head

<bound method DataFrame.head of            dataid                 city  local2  tech2  eponymous  is_corp  \
0         1873685              ARCADIA       0      0          0        1   
1         1873686            SAN DIEGO       0      0          0        1   
2         1873687     WESTLAKE VILLAGE       0      0          0        1   
3         1873688           SAN MARCOS       0      0          0        1   
4         1873689            SAN DIEGO       0      0          0        1   
5         1873690              OAKLAND       0      0          0        1   
6         1873691             VALENCIA       0      0          0        1   
7         1873693          WESTMINSTER       0      0          0        1   
8         1873694       SAN BERNARDINO       1      0          1        1   
9         1873695             LA JOLLA       0      0          0        1   
10        1873696          CULVER CITY       1      0          1        1   
11        1873697            ELK GROVE      

In [634]:
#label_names = ['eponymous','is_corp','local2','tech2','words_1or2','is_DE']
#label_names= ['trademark','anypatent']
label_names = ['eponymous','is_corp','local2','tech2','words_1or2','trademark','only_patent','only_DE','patent_and_DE']
#data_reg = data[label_names + ['sampletrain']]
#labels_reg = data[['growthz'] + ['sampletrain']]
out_name = 'growthz';

data_train_feat = data[(data.incyear <= 2006) & (data.sampletrain == 1)]
data_train_feat = data_train_feat[label_names]

data_test_feat = data[(data.incyear <= 2006) & (data.sampletrain == 0)]
data_test_feat = data_test_feat[label_names]

data_eval_feat = data[(data.incyear > 2006)]
data_eval_feat = data_eval_feat[label_names]


data_train_labels = data[(data.incyear <= 2006) & (data.sampletrain == 1)]
data_train_labels = data_train_labels[out_name]

data_test_labels = data[(data.incyear <= 2006) & (data.sampletrain == 0)]
data_test_labels = data_test_labels[out_name]

data_eval_labels = data[(data.incyear > 2006)]
data_eval_labels = data_eval_labels[out_name]

print data_train_feat.describe()
#print "-----------------------------------------------------------------------------------"
#print data_eval_feat.describe()
print "-----------------------------------------------------------------------------------"
print data_train_labels

           eponymous        is_corp         local2          tech2  \
count  585162.000000  585162.000000  585162.000000  585162.000000   
mean        0.100348       0.626117       0.152524       0.006545   
std         0.300464       0.483833       0.359528       0.080637   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       1.000000       0.000000       0.000000   
75%         0.000000       1.000000       0.000000       0.000000   
max         1.000000       1.000000       1.000000       1.000000   

          words_1or2      trademark    only_patent        only_DE  \
count  585162.000000  585162.000000  585162.000000  585162.000000   
mean        0.134715       0.006855       0.003512       0.043514   
std         0.341419       0.082508       0.059157       0.204012   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.0000

In [623]:
from sklearn import cross_validation, grid_search, linear_model, metrics
from sklearn.utils import column_or_1d
#from sklearn import grid_search
#from sklearn import linear_model
import numpy as np

features_train = data_train_feat.values
features_test = data_test_feat.values
features_eval = data_eval_feat.values
labels_train = np.ravel(data_train_labels.values)
labels_test = np.ravel(data_test_labels.values)
labels_eval = np.ravel(data_eval_labels.values)

#features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(
#    data_train_feat.values, np.ravel(data_train_labels.values), test_size=0.3, random_state=42)

In [633]:
import statsmodels.api as sm

logitModel = sm.Logit(labels_train,sm.add_constant(features_train, prepend=False))
logitResult = logitModel.fit()

print logitResult.summary()

Optimization terminated successfully.
         Current function value: 0.003292
         Iterations 14
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:               585162
Model:                          Logit   Df Residuals:                   585152
Method:                           MLE   Df Model:                            9
Date:                Tue, 15 Sep 2015   Pseudo R-squ.:                  0.3224
Time:                        14:32:29   Log-Likelihood:                -1926.5
converged:                       True   LL-Null:                       -2843.3
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1            -1.4317      0.509     -2.812      0.005        -2.430    -0.434
x2             1.8111      0

In [602]:
print np.mean(logitResult.predict(sm.add_constant(features_eval, prepend=False)))
print np.std(logitResult.predict(sm.add_constant(features_eval, prepend=False)))
print np.percentile(logitResult.predict(sm.add_constant(features_eval, prepend=False)),99)

0.000606785369558
0.00602014562795
0.00908859974132


In [286]:
#exponents = np.linspace(0,10,11)
#print [10**x for x in exponents] 
#parameters = {'C':[1, 10]}
#clf = linear_model.LogisticRegressionCV(Cs=[10**x for x in exponents])
clf = linear_model.LogisticRegression(C=10000000,dual=False,solver='lbfgs')
clf.fit(features_train,labels_train)

#clf.fit(data_train_feat.values, np.ravel(data_train_labels.values))
#print clf.C_

LogisticRegression(C=10000000, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0)

In [563]:
count = 0;
for name in label_names:
    print name," ", clf.coef_[0][count]
    count += 1

print "Intercept ",clf.intercept_[0]
print "-----------------------------------------------------------------------------------"

print "Score ",metrics.r2_score(clf.predict_proba(features_test)[:,1],labels_test)

eponymous   -1.43188581483
words_1or2   0.408266807774
local2   -0.97169595215
tech2   0.651534904643
is_corp   1.81110554763
trademark   1.67973944928
only_patent   3.21988681708
only_DE   3.58143932047
patent_and_DE   5.27559264479
Intercept  -10.1308618376
-----------------------------------------------------------------------------------
Score  -19.1725122834


In [606]:
prob_test = logitResult.predict(sm.add_constant(features_test, prepend=False))#clf.predict_proba(features_test)
probDataTest = pandas.DataFrame(prob_test)#(prob_test[:,1])

print "-----------------------------------------------------------------------------------"
print probDataTest.describe()
print probDataTest.median()

featsLabelsTest = np.array([prob_test,labels_test])
sortedFeats = featsLabelsTest[:,np.argsort(-1*featsLabelsTest[0])]
topFive = sortedFeats[:,1:round(0.05*len(sortedFeats[0]))]
print "-----------------------------------------------------------------------------------"
print 100*(topFive.sum(axis=1)/sortedFeats.sum(axis=1))[1]

topOne = sortedFeats[:,1:round(0.01*len(sortedFeats[0]))]
print "-----------------------------------------------------------------------------------"
print 100*(topOne.sum(axis=1)/sortedFeats.sum(axis=1))[1]

-----------------------------------------------------------------------------------
                   0
count  250784.000000
mean        0.000568
std         0.005570
min         0.000015
25%         0.000040
50%         0.000092
75%         0.000244
max         0.328951
0    0.000092
dtype: float64
-----------------------------------------------------------------------------------
76.3636363636
-----------------------------------------------------------------------------------
56.3636363636


In [607]:
features_all = np.concatenate((features_test, features_train, features_eval))

prob = logitResult.predict(sm.add_constant(features_eval, prepend=False))#clf.predict_proba(features_eval)
probData = pandas.DataFrame(prob)
print probData.describe()
print probData.median()

probDataOnePerc = -1.0*probData.values
print "-----------------------------------------------------------------------------------"
print probDataOnePerc
probDataOnePerc.sort(axis=0)
print "-----------------------------------------------------------------------------------"
print probDataOnePerc
topOne = pandas.DataFrame(-1*probDataOnePerc[1:round(0.01*len(features_eval))])
print "-----------------------------------------------------------------------------------"
print topOne.describe()
print topOne.median()

                   0
count  754424.000000
mean        0.000607
std         0.006020
min         0.000015
25%         0.000040
50%         0.000060
75%         0.000244
max         0.424434
0    0.00006
dtype: float64
-----------------------------------------------------------------------------------
[[ -2.43586875e-04]
 [ -2.43586875e-04]
 [ -3.66386102e-04]
 ..., 
 [ -3.98273914e-05]
 [ -3.98273914e-05]
 [ -3.98273914e-05]]
-----------------------------------------------------------------------------------
[[ -4.24434050e-01]
 [ -4.24434050e-01]
 [ -4.24434050e-01]
 ..., 
 [ -1.50729302e-05]
 [ -1.50729302e-05]
 [ -1.50729302e-05]]
-----------------------------------------------------------------------------------
                 0
count  7543.000000
mean      0.034267
std       0.048783
min       0.009089
25%       0.012997
50%       0.012997
75%       0.045474
max       0.424434
0    0.012997
dtype: float64


In [608]:
population_data = pandas.read_stata("population.dta")
cities = population_data['city'].values
data_eval_feat_city = data[(data.incyear > 2006)]
data_eval_feat_city = pandas.DataFrame({'city': data_eval_feat_city[['city']].values[:,0],
                                        'growthz' : probData.values[:,0]})#pandas.concat([data_eval_feat_city[['city']], probData],axis=1)

print data_eval_feat_city.describe()

             growthz
count  754424.000000
mean        0.000607
std         0.006020
min         0.000015
25%         0.000040
50%         0.000060
75%         0.000244
max         0.424434


In [609]:
temp = []
data_city_clean = data_eval_feat_city[data_eval_feat_city['city'].isin(cities)]
print data_city_clean
#for city in cities:
#    tempFrame = data_eval_feat_city[data_eval_feat_city.city == city]
#    temp.append(tempFrame.mean())

                          city   growthz
0                     SAN JOSE  0.000244
1                    PALO ALTO  0.000244
2                      HAYWARD  0.000366
3                     GLENDALE  0.000244
4                       IRVINE  0.000244
5                    CUPERTINO  0.000244
7                       TUSTIN  0.000244
8                       IRVINE  0.000244
9                     MILLBRAE  0.000244
10                     ARCADIA  0.000244
11                 LOS ANGELES  0.000244
12                    SAN JOSE  0.000092
13                      TUSTIN  0.000244
14                  LONG BEACH  0.000244
15                   LANCASTER  0.000366
16                    SAN JOSE  0.000244
17                      TUSTIN  0.000366
18                  LONG BEACH  0.000244
19                      FRESNO  0.000244
20                      GOLETA  0.000092
21                        NAPA  0.000244
22                    WILDOMAR  0.000092
23                    TORRANCE  0.000244
24              

In [610]:
print data_city_clean.groupby(['city']).get_group('PALO ALTO').describe()
print "-----------------------------------------------------------------------------------"
print data_city_clean.groupby(['city']).get_group('MOUNTAIN VIEW').describe()
print "-----------------------------------------------------------------------------------"
print data_city_clean.groupby(['city']).mean().describe()
print data_city_clean.groupby(['city']).mean().quantile(0.99)

print "-----------------------------------------------------------------------------------"

           growthz
count  4302.000000
mean      0.004396
std       0.016052
min       0.000015
25%       0.000040
50%       0.000244
75%       0.001429
max       0.277671
-----------------------------------------------------------------------------------
           growthz
count  2106.000000
mean      0.004447
std       0.016999
min       0.000015
25%       0.000040
50%       0.000244
75%       0.000467
max       0.277671
-----------------------------------------------------------------------------------
          growthz
count  482.000000
mean     0.000437
std      0.000593
min      0.000051
25%      0.000160
50%      0.000234
75%      0.000451
max      0.004637
growthz    0.003452
dtype: float64
-----------------------------------------------------------------------------------


In [611]:
import operator

val = np.reshape(data_city_clean.groupby(['city']).mean().values,len(data_city_clean.groupby(['city']).groups.keys()))
byCity = np.reshape(np.array([data_city_clean.groupby(['city']).groups.keys()]),len(data_city_clean.groupby(['city']).groups.keys()))

cityData = data_city_clean.groupby(['city']).mean().apply(lambda x:1000*x)
print cityData.sort(columns=['growthz'], ascending=False)
#cityDict.update({n: np.mean(cityDict[n]) for n in cityDict.keys()})

#sortedcityDict = sorted(cityDict.items(), key=operator.itemgetter(1), reverse=True)

#print sortedcityDict

#idxs = np.argsort(-1*val)
#for idx in idxs:
#    print byCity[idx],"\t",val[idx]
#sortedByCity = byCity[np.argsort(-1*val)]
#sortedVal = 1000*val[np.argsort(-1*val)]
#print sortedByCity, sortedVal
#sortedFeats = featsLabelsTest[:,np.argsort(-1*featsLabelsTest[0])]

                      growthz
city                         
MENLO PARK           4.636908
MOUNTAIN VIEW        4.447135
PALO ALTO            4.396403
SUNNYVALE            3.858401
REDWOOD CITY         3.784624
EAST PALO ALTO       3.373497
EMERYVILLE           3.250754
PORTOLA VALLEY       2.680680
GROVER BEACH         2.424233
SAN MATEO            2.330189
SOUTH SAN FRANCISCO  2.278259
LOS ALTOS HILLS      2.260529
LOS ALTOS            2.224483
WOODSIDE             2.125992
GOLETA               2.114936
SANTA CLARA          2.056838
FOSTER CITY          1.978786
CUPERTINO            1.873815
SCOTTS VALLEY        1.737829
SAN FRANCISCO        1.649697
BURLINGAME           1.614657
CARMEL-BY-THE-SEA    1.588041
TRINIDAD             1.505401
FREMONT              1.468877
SAN BRUNO            1.464017
LARKSPUR             1.435745
ATHERTON             1.368214
BELVEDERE            1.355766
SAN CARLOS           1.298759
ALISO VIEJO          1.283176
...                       ...
SOLEDAD   